In [1]:
import os
import numpy as np
import cv2
import re
from skimage import io, feature, morphology, segmentation, measure, filters, exposure
from scipy.ndimage import distance_transform_edt
import matplotlib.pyplot as plt
from skimage.morphology import disk, dilation,  skeletonize
from skimage.filters import gaussian
from scipy.signal import convolve2d
from scipy import ndimage as ndi,ndimage
from sklearn.decomposition import PCA

In [2]:
# Define input image path and output folder
input_folder = r"C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\segmented_with_SAM"
output_folder = r"C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extraction_dis_x_ske_SAM"

In [3]:
def moving_average(data, window_size=3):
    return np.convolve(data, np.ones(window_size)/window_size, mode='same')

In [4]:
#funtion to split the images
def split_region(image, x, y, w, h, min_size, std_thresh):
    region = image[y:y+h, x:x+w]

    if w <= min_size or h <= min_size or is_homogeneous(region, std_thresh):
        return [(x, y, w, h)]

    hw, hh = w // 2, h // 2
    regions = []
    regions += split_region(image, x, y, hw, hh, min_size, std_thresh)
    regions += split_region(image, x + hw, y, w - hw, hh, min_size, std_thresh)
    regions += split_region(image, x, y + hh, hw, h - hh, min_size, std_thresh)
    regions += split_region(image, x + hw, y + hh, w - hw, h - hh, min_size, std_thresh)

    return regions

In [5]:
def score_closing(binary_mask, kernel_size, target_regions=2):
    structure = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    closed_mask = ndimage.binary_closing(binary_mask, structure=structure)

    # Label connected regions
    labeled, num_features = ndimage.label(closed_mask)
    
    # Score: absolute distance to desired number of objects
    score = abs(num_features - target_regions)

    return score, closed_mask, num_features

In [6]:
def extract_number(filename):
    # Extract number from "download(43).png"
    match = re.search(r'\((\d+)\)', filename)
    return int(match.group(1)) if match else -1

In [7]:
# Get sorted list of PNG files based on the number inside parentheses
sorted_files = sorted(
    [f for f in os.listdir(input_folder) if f.endswith((".png", ".jpg", ".jpeg"))],
    key=extract_number
)

In [8]:
# Process all images
for filename in sorted_files:
    image_path = os.path.join(input_folder, filename)
    os.makedirs(output_folder, exist_ok=True)

    # Load image
    img = io.imread(image_path)

    # Pad image
    padded_image = cv2.copyMakeBorder(
        img,
        top=40, bottom=40, left=40, right=40,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )

    # Convert to grayscale
    padded_mask = cv2.cvtColor(padded_image, cv2.COLOR_RGB2GRAY)
    
    # Normalize mask
    scale = 1000 / max(padded_mask.shape)
    resized_mask = cv2.resize(padded_mask, None, fx=scale, fy=scale, interpolation=cv2.INTER_NEAREST)

    # Find best kernel size for closing
    best_score = float('inf')
    best_kernel = 3
    for k in range(3, 20, 2):  # Odd sizes only
        score, result_mask, num_regions = score_closing(resized_mask, k)
        if score < best_score:
            best_score = score
            best_mask = result_mask
            best_kernel = k

    # Apply best kernel for morphological cleaning
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (best_kernel, best_kernel))
    mask_clean = cv2.morphologyEx(resized_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Convert to uint8 for distance transform
    binary_mask_uint8 = (mask_clean * 255).astype(np.uint8)

    # Distance transform
    dist_transform = cv2.distanceTransform(binary_mask_uint8, cv2.DIST_L2, 5)

    # Skeletonize
    skeleton = skeletonize(mask_clean).astype(np.float32)

    # Multiply to get width map
    new_object =  skeleton * dist_transform  
    full_width = new_object*2

    # Extract row-wise max widths
    row_max_widths = []
    for y in range(full_width.shape[0]):
        x_indices = np.where(full_width[y] > 0)[0]
        if len(x_indices) > 0:
            widths =  full_width[y, x_indices]
            row_max_widths.append(np.max(widths))
    smoothed_data = np.insert(row_max_widths, 0, 0)        
    iterations = 10
    for i in range(iterations):
        smoothed_data = np.insert(smoothed_data, 0, 0)
        smoothed_data = moving_average(smoothed_data, window_size=5)
    smoothed_data = smoothed_data[:200]
    # Save widths to file
    output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_widths.txt")
    with open(output_file, "w") as file:
        for width in smoothed_data:
            file.write(f"{int(width)}\n")

    print(f"Widths saved to {output_file}")

Widths saved to C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extraction_dis_x_ske_SAM\download (0)_widths.txt
Widths saved to C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extraction_dis_x_ske_SAM\download (1)_widths.txt
Widths saved to C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extraction_dis_x_ske_SAM\download (2)_widths.txt
Widths saved to C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extraction_dis_x_ske_SAM\download (3)_widths.txt
Widths saved to C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extraction_dis_x_ske_SAM\download (4)_widths.txt
Widths saved to C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extraction_dis_x_ske_SAM\download (5)_widths.txt
Widths saved to C:\Users\Paolo\OneDrive\Desktop\Thesis\Mycos_old_data (1)\Mycos_old_data\2005\width_extrac